<a href="https://colab.research.google.com/github/lukriste/Baigiamasis/blob/lenteles_tvarkymas/modelioT5_treniravimas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paruošimas

In [15]:
from google.colab import files
uploaded = files.upload()

Saving naujas_gnm.xlsx to naujas_gnm (1).xlsx


In [16]:
df = pd.read_excel("naujas_gnm.xlsx")

In [24]:
import pandas as pd
from sqlalchemy import create_engine
import re

In [45]:

df = df.rename(columns={
    "Siptomas": "simptomas",
    "Input/situacija": "ivestis",
    "Output/GNM_interpretacija": "isvestis"
})

def normalizuoti_teksta(tekstas):
    if pd.isna(tekstas):
        return ""
    tekstas = str(tekstas).strip()
    if tekstas.startswith('"') and tekstas.endswith('"'):
        tekstas = tekstas[1:-1]
    tekstas = tekstas.replace("“", '"').replace("”", '"').replace("„", '"')
    tekstas = tekstas.replace("‘", "'").replace("’", "'")
    tekstas = tekstas.replace("–", "-").replace("—", "-")
    tekstas = tekstas.replace("\\", "/")

    if tekstas.startswith('"'):
        tekstas = tekstas[1:]
    if tekstas.endswith('"'):
        tekstas = tekstas[:-1]

    tekstas = re.sub(r"[\r\n\t]", " ", tekstas)
    tekstas = re.sub(r"\s{2,}", " ", tekstas)
    tekstas = re.sub(r"\s+([.,;!?()])", r"\1", tekstas)
    return tekstas

df["simptomas"] = df["simptomas"].apply(lambda x: normalizuoti_teksta(x).lower())
df["ivestis"] = df["ivestis"].apply(lambda x: normalizuoti_teksta(x).lower())
df["isvestis"] = df["isvestis"].apply(normalizuoti_teksta)

df["ivestis"] = "simptomas: " + df["simptomas"] + ". " + df["ivestis"]

In [46]:
!pip install transformers datasets sentencepiece

In [47]:
!pip install openpyxl pymysql sqlalchemy

In [48]:
from datasets import Dataset
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

Tokenizavimo funkcija

In [49]:
from transformers import T5Tokenizer, TFAutoModelForSeq2SeqLM
import tensorflow as tf

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

In [50]:
def tokenize_function(example):
    input_enc = tokenizer(example["ivestis"], padding="max_length", truncation=True, max_length=128)
    output_enc = tokenizer(example["isvestis"], padding="max_length", truncation=True, max_length=128)
    input_enc["labels"] = output_enc["input_ids"]
    return input_enc

train_tokenized = train_dataset.map(tokenize_function)
test_tokenized = test_dataset.map(tokenize_function)


Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

TF dataset paruošimas

In [51]:
train_set = train_tokenized.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["labels"],
    shuffle=True,
    batch_size=8,
)

val_set = test_tokenized.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["labels"],
    shuffle=False,
    batch_size=8,
)


 Modelio treniravimas

In [54]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5))
history = model.fit(train_set, validation_data=val_set, epochs=100)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Epoch 1/100
12/12 [==============================] - 37s 520ms/step - loss: 5.8201 - val_loss: 4.5707
Epoch 2/100
12/12 [==============================] - 2s 177ms/step - loss: 4.4623 - val_loss: 3.7230
Epoch 3/100
12/12 [==============================] - 2s 172ms/step - loss: 3.8552 - val_loss: 3.5760
Epoch 4/100
12/12 [==============================] - 2s 180ms/step - loss: 3.7242 - val_loss: 3.4716
Epoch 5/100
12/12 [==============================] - 2s 185ms/step - loss: 3.5982 - val_loss: 3.3869
Epoch 6/100
12/12 [==============================] - 2s 177ms/step - loss: 3.4968 - val_loss: 3.3143
Epoch 7/100
12/12 [==============================] - 2s 174ms/step - loss: 3.4297 - val_loss: 3.2496
Epoch 8/100
12/12 [==============================] - 2s 180ms/step - loss: 3.4043 - val_loss: 3.1875
Epoch 9/100
12/12 [==============================] - 2s 176ms/step - loss: 3.3176 - val_loss: 3.1239
Epoch 10/100
12/12 [==============================] - 2s 196ms/step - loss: 3.2804 - val_l

 8. Modelio testavimas

In [55]:
model.save_pretrained("gnm-t5-final")
tokenizer.save_pretrained("gnm-t5-final")

('gnm-t5-final/tokenizer_config.json',
 'gnm-t5-final/special_tokens_map.json',
 'gnm-t5-final/spiece.model',
 'gnm-t5-final/added_tokens.json')

In [71]:
def test_ivestis(ivestis):
    input_ids = tokenizer(ivestis, return_tensors="tf", truncation=True, padding=True).input_ids
    output_ids = model.generate(
        input_ids,
        max_length=128,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=2.5,
        eos_token_id=tokenizer.eos_token_id,
        decoder_start_token_id=tokenizer.pad_token_id
    )
    isvestis = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"🔹 Įvestis: {ivestis}")
    print(f"🧠 Sugeneruota išvestis: {isvestis}")
    return isvestis

test_ivestis("insultas. žmogų ištiko insultas po skyrybų.")

🔹 Įvestis: insultas. žmogų ištiko insultas po skyrybų.
🧠 Sugeneruota išvestis: mog itiko insultas po skyrybimu. Tai insultacimou svarvard interesu, kad fresco gali bti susij su nenoru pasireikio danikumo konflikto del reakcija innovativistu!


'mog itiko insultas po skyrybimu. Tai insultacimou svarvard interesu, kad fresco gali bti susij su nenoru pasireikio danikumo konflikto del reakcija innovativistu!'